# From Baseline CNN to Deeper Architectures

### **1. Introduction: From Baseline CNN to Deeper Architectures**

In the previous notebook (NB03), we trained a **baseline CNN** on the Oxford-IIIT Pet dataset and analyzed its performance. While it achieved **~13% accuracy** — slightly above random guessing for 37 breeds — our detailed error analysis revealed key shortcomings:

- The network relied heavily on **color cues** (e.g., black animals predicted as Bombay, white animals as Samoyed).  
- It struggled with **fine-grained distinctions** between visually similar breeds (e.g., Chihuahua vs. Miniature Pinscher).  
- It often confused **cats and dogs**, failing to separate species-level features in some cases.

These issues highlight a fundamental limitation: the baseline CNN is **too shallow** to build the hierarchical representations (edges → textures → parts → objects) needed for fine-grained breed classification.


**What We’ll Do in NB04**

In this notebook, we address these limitations by:

- **Introducing deeper architectures** (e.g., LeNet, AlexNet-inspired models) that can capture more complex patterns.  
- **Comparing performance and parameter efficiency** across models to see how depth affects generalization.  
- Revisiting the concept of **feature hierarchies** to understand *why* deeper networks tend to perform better on visual tasks.

By the end of this notebook, we’ll have a clearer understanding of how **model depth** impacts learning and be prepared to explore **transfer learning** and more advanced architectures in future notebooks.

---

### **2. Reload Data & Baseline Model**

In this notebook, we compare multiple CNN architectures (Baseline, LeNet-like, AlexNet-mini).  
To ensure **self-containment**, we **reload the dataset and baseline model weights** here — independent of NB02.

**Key Points:**

- **Dataset:** Oxford-IIIT Pets (37 breeds), resized to **128×128**, normalized to `[-1,1]` (mean/std ≈ 0.5).
- **Splits:** 
  - Training/Validation: 80/20 split from `trainval`.
  - Test: Provided separately.
- **Device:** CPU-only.
- **Baseline Model:** Reload weights trained in NB02 for direct comparison.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# -----------------------------
# 1. CPU-Only Device Setup
# -----------------------------
device = 'cpu'

IMG_SIZE = 128
transform = transforms.Compose(
    [transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])]
)

# -----------------------------
# 3. Load Train/Val/Test Datasets
# -----------------------------
data_path = "./data/"

trainval_dataset = datasets.OxfordIIITPet(
    root = data_path,
    split = 'trainval',
    target_types = 'category',
    download = True,
    transform = transform
)

test_dataset = datasets.OxfordIIITPet(
    root = data_path,
    split = 'test',
    target_types = 'category',
    download = True,
    transform = transform
)

# -----------------------------
# 4. Train/Val Split (80/20)
# -----------------------------
split_size = 0.8
train_size = int(len(trainval_dataset) * split_size)
val_size = len(trainval_dataset) - train_size

train_dataset, val_dataset = random_split(
    trainval_dataset, 
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
    )

# -----------------------------
# 5. DataLoaders
# -----------------------------
BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


# Confirm splits
print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")
print(f"Number of classes: {len(trainval_dataset.classes)}")


Train size: 2944
Validation size: 736
Test size: 3669
Number of classes: 37


In [3]:
# -----------------------------
# 6. Baseline CNN Definition (same as NB02)
# -----------------------------
class_names = trainval_dataset.classes
class PetCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, len(class_names))
    
    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        # flatten
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate and load baseline weights (map to CPU)
baseline_model = PetCNN().to(device)
baseline_model.load_state_dict(torch.load("petcnn_best.pth", map_location=device, weights_only=False))

print("Baseline CNN loaded successfully (CPU-only). Ready for comparison.")


Baseline CNN loaded successfully (CPU-only). Ready for comparison.


---

**✅ Code Task 5.4.2.1: Create a Small Train/Val Split & DataLoaders**

📘 Instruction</br>
For quick experiments on CPU, let’s create a small custom split from trainval_dataset and build DataLoaders.

- Use random_split with a fixed seed for reproducibility.
- Make the split sizes student-chosen (e.g., ~800 for train, ~200 for val), but fill in the ....
- Build DataLoaders with batch_size=16 and shuffle=True for train.
- Print the sizes and store one batch’s tensor shape in CT_batch_shape.

In [4]:
from torch.utils.data import random_split, DataLoader

# small custom split for quick CPU experiments
CT_total = len(trainval_dataset)
CT_train_size = int(CT_total * 0.8)
CT_val_size = CT_total - CT_train_size
CT_train_dataset, CT_val_dataset = random_split(
    trainval_dataset,
    [CT_train_size, CT_val_size],
    generator=torch.Generator().manual_seed(123)
)

CT_BATCH_SIZE = 16
CT_train_loader = DataLoader(CT_train_dataset, batch_size=CT_BATCH_SIZE, shuffle=True)
CT_val_loader   = DataLoader(CT_val_dataset,   batch_size=CT_BATCH_SIZE, shuffle=False)

# Peek one batch and record its shape (N, C, H, W)
CT_images, CT_labels = next(iter(CT_train_loader))
CT_batch_shape = CT_images.shape
print("CT_train/val sizes:", len(CT_train_dataset), len(CT_val_dataset))
print("CT_batch_shape:", CT_batch_shape)


CT_train/val sizes: 2944 736
CT_batch_shape: torch.Size([16, 3, 128, 128])


### **3. Architectural Variants Overview (Concept)**

In this notebook, we **compare architectures** to explore how **depth and design choices** affect performance and parameter efficiency.

**Architectures We’ll Compare**

1. **Baseline CNN (NB02)**  
   - 2 convolution layers (3×3) + 2 fully connected layers.  
   - Designed for teaching fundamentals; very shallow.

2. **LeNet-like CNN (1998)**  
   - Classic architecture for MNIST digit recognition.  
   - 3 convolution layers + 2 fully connected layers.  
   - Slightly deeper; captures more **mid-level features** (edges → textures).

3. **AlexNet-mini (2012) / VGG-lite**  
   - Inspired by ImageNet-winning networks.  
   - 4–5 convolution layers + deeper FC layers.  
   - Captures **hierarchical features** (edges → textures → object parts → objects).

**High-Level ASCII Diagrams**

**(a) Baseline CNN (NB02)**

```text
Input (3×128×128)
│
├── Conv(3→16) → ReLU → MaxPool(2×2)
│
├── Conv(16→32) → ReLU → MaxPool(2×2)
│
├── Flatten → FC(32×32×32 → 128) → ReLU
│
└── FC(128 → 37) → Softmax
```

**(b) LeNet-like CNN**

```text
Input (3×128×128)
│
├── Conv(3→16) → ReLU → MaxPool(2×2)
│
├── Conv(16→32) → ReLU → MaxPool(2×2)
│
├── Conv(32→64) → ReLU → MaxPool(2×2)
│
├── Flatten → FC(64×16×16 → 256) → ReLU
│
└── FC(256 → 37) → Softmax
```

**(c) AlexNet-mini / VGG-lite**

```text
Input (3×128×128)
│
├── Conv(3→32) → ReLU → Conv(32→32) → ReLU → MaxPool(2×2)
│
├── Conv(32→64) → ReLU → Conv(64→64) → ReLU → MaxPool(2×2)
│
├── Conv(64→128) → ReLU → MaxPool(2×2)
│
├── Flatten → FC(128×16×16 → 512) → ReLU → Dropout
│
└── FC(512 → 37) → Softmax

```

**Parameter Count and Shape Transformations**

| Layer Type    | Baseline CNN                  | LeNet-like           | AlexNet-mini          |
|---------------|-------------------------------|----------------------|-----------------------|
| **Conv Layers** | Few params ($3\times3$ filters) | More filters (16→64) | Many filters (32→128) |
| **FC Layers**   | Dominant ($32k \to 128$)       | Moderate ($64\times16\times16 \to 256$) | Larger ($128\times16\times16 \to 512$) |
| **Total Params**| ~ 4$M                      | ~ 6$M            | ~ 10$–$12$M       |

**Receptive Field Expansion**

- **Baseline CNN**: Each neuron “sees” only small local patches.
- **LeNet-like**: Deeper layers combine local features → larger effective receptive field.
- **AlexNet-mini**: Stacks more conv layers; receptive field covers most of the image, capturing global context (e.g., full face of a dog).


**Why Compare These?**

- **Highlights trade-offs:**
  - Depth vs. computation (more convs = more params but richer features).
  - Pooling strategy (aggressive vs gradual downsampling).
  - Fully connected size (impact on parameter efficiency).

- **Prepares us for Project 6:**
  - Introduction to transfer learning (pretrained deeper networks like **ResNet**).


---

### **4. Implementing Architectural Variants (Code)**

We now translate the three conceptual architectures into PyTorch models:

1. **Baseline CNN** (from NB02)  
   - 2 convolution layers → pooling → 2 fully connected layers.

2. **LeNet-like CNN** (slightly deeper)  
   - 3 convolution layers → pooling after first two → 2 fully connected layers.  
   - Inspired by **LeNet-5 (1998)** but adapted to RGB and 128×128 inputs.

3. **AlexNet-mini (VGG-lite)**  
   - 5 convolution layers grouped in blocks (Conv → ReLU → Conv → ReLU → Pool).  
   - Deeper feature hierarchy; mimics **AlexNet/VGG-style** stacking but scaled down for Pets dataset.

We keep implementation **consistent**:
- Use `nn.Conv2d`, `nn.ReLU`, `nn.MaxPool2d` for feature extraction.
- Use `nn.Linear` for classification layers.
- Forward pass includes **comments on shape changes** (C×H×W).


In [17]:
# -------------------------------
# 1. Baseline CNN (for reference)
# -------------------------------

from turtle import forward


class BaselineCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 37)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Flattening Layer
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# -------------------------------
# 2. LeNet-like CNN (3 conv layers)
# -------------------------------
class LeNetLikeCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        # Pooling Layer
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, len(class_names))
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flattening Layers
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# -------------------------------
# 3. AlexNet-mini CNN (5 conv layers)
# -------------------------------
class AlexNetMini(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolution layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1) #  3 x 128 x 128 -> 32 x 128 x 128
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 32 x 64 x 64 -> 64 x 64 x 64
        self.conv3  = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 37)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))

        # Flattening
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In the following section, we will:
- **Instantiate each model**.
- Compute and compare **parameter counts** (baseline vs LeNet-like vs AlexNet-mini).
- Discuss trade-offs: **depth vs parameter efficiency vs performance**.

---

**✅ Code Task 5.4.4.1: Implement a LeNet Variant (with Dropout/BatchNorm)**

📘 Instruction</br>
Implement CT_LeNetVariant as a LeNet-like model with 3 conv blocks, BatchNorm after each conv, and Dropout before the final classifier. Use this spec:
- conv1: 3→16, conv2: 16→32, conv3: 32→64, each kernel_size=3, padding=1, followed by ReLU, BatchNorm2d, and MaxPool2d(2).
- After three pools, spatial dims: 128 → 64 → 32 → 16, so flatten size: 64*16*16.
- fc1: 64*16*16 → 256 (ReLU), Dropout(p=0.5), fc2: 256 → 37.</br>
Instantiate as CT_model = CT_LeNetVariant().to(device).

In [18]:
import torch.nn as nn
import torch.nn.functional as F

class CT_LeNetVariant(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(16)
        self.pool  = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(64)

        self.fc1   = nn.Linear(64 * 16 * 16, 256)   # 64*16*16 -> 256
        self.drop  = nn.Dropout(p=0.5)
        self.fc2   = nn.Linear(256, 37)   # 256 -> 37

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x

CT_model = CT_LeNetVariant().to(device)
print(CT_model)


CT_LeNetVariant(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=16384, out_features=256, bias=True)
  (drop): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=37, bias=True)
)


**✅ Code Task 5.4.4.2: Param Count & Forward-Shape Sanity Check**

📘 Instruction</br>
Write a helper to count parameters and do a single forward pass on one batch from CT_train_loader to verify output shape (N, 37).
Store:
- CT_n_params (int): total learnable parameters of CT_model
- CT_logits (tensor): output logits from a single batch
- Print both.

In [19]:
def CT_count_params(model):
    n_params = sum(p.numel() for m in model.modules() if isinstance(m, nn.Linear) for p in m.parameters() if p.requires_grad)
    return n_params

CT_model.eval()
with torch.no_grad():
    CT_x, CT_y = next(iter(CT_train_loader))
    CT_x = CT_x.to(device)
    CT_logits = CT_model(CT_x)
    CT_n_params = CT_count_params(CT_model)

print("CT_logits shape:", CT_logits.shape)
print("CT_n_params:", CT_n_params)

CT_logits shape: torch.Size([16, 37])
CT_n_params: 4204069


---

### **5. Training Strategy**

We now train and compare **three CNN architectures** of increasing depth:

1. **Baseline CNN** – 2 convolution layers (from NB02).
2. **LeNet-like CNN** – 3 convolution layers (moderate depth).
3. **AlexNet-mini (VGG-lite)** – 5 convolution layers (deepest).

**Fair Comparison Setup**

To ensure results are comparable:

- **Dataset:** Oxford-IIIT Pets (128×128, normalized to `[-1, 1]`)
- **Hardware:** CPU-only
- **Hyperparameters (same for all models):**
  - Optimizer: `Adam` (learning rate = 0.001)
  - Loss: `CrossEntropyLoss`
  - Batch size: 32 
  - Epochs: 5

**Parameter Awareness**

- We print **parameter counts** for each model *before training*.
- This helps relate **model depth vs parameter count vs performance** later.
- Expect:
  - **Baseline**: small, fast to train.
  - **LeNet-like**: slightly deeper, similar parameter count.
  - **AlexNet-mini**: deepest, more capacity but also risk of overfitting.

**Training Process**

- Reuse modular `train_model()` function from NB02.
- Train models sequentially (Baseline → LeNet-like → AlexNet-mini).
- Save **best model weights** for each:
  - `baseline_best.pth`
  - `lenet_best.pth`
  - `alexmini_best.pth`

Now let’s show how we'd run the training and record parameter counts. 


#**DISCLAIMER**: We are simulating the training process and using from now on the pre-trained models that you can see in the notebook files. This means that although you have the code we've used to train the model, we are not running the training loops here; otherwise, the VM would collapse and, more importantly, we wouldn't be able to provide this completely free lab for thousands of students. However, you can try to train the model on other platforms, such as Google Colab.

In [20]:
import time
import copy


def calculate_accuracy(outputs, labels):
    _, preds = torch.argmax(outputs, 1)
    return (preds == labels).sum().item() / len(labels)


# Training function (CPU-only)
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=5, device='cpu'):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    model.to(device)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print('-' * 20)

        # Train and validation phases
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                loader = train_loader
            else:
                model.eval()
                loader = val_loader

            running_loss = 0.0
            running_corrects = 0
            total_samples = 0

            for inputs, labels in loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += (outputs.argmax(1) == labels).sum().item()
                total_samples += labels.size(0)

            epoch_loss = running_loss / total_samples
            epoch_acc = running_corrects / total_samples

            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accuracies.append(epoch_acc)
            else:
                val_losses.append(epoch_loss)
                val_accuracies.append(epoch_acc)

                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")
    print(f"Best val loss: {best_loss:.4f}")

    # Load best weights
    model.load_state_dict(best_model_wts)

    return model, (train_losses, val_losses, train_accuracies, val_accuracies)


In [21]:
# parameter count
def count_params(model):
    return sum(p.numel() for p in model.parameters())

# Instantiate models
baseline_model = BaselineCNN()
lenet_model = LeNetLikeCNN()
alexmini_model = AlexNetMini()

# Print parameter counts
print(f"BaselineCNN parameters: {count_params(baseline_model):,}")
print(f"LeNetLikeCNN parameters: {count_params(lenet_model):,}")
print(f"AlexNetMini parameters: {count_params(alexmini_model):,}")

# Define common loss and optimizer settings
criterion = nn.CrossEntropyLoss()

# Train Baseline
print("\n= Training Baseline CNN (we are using pre-trainned models) =")
# Train LeNet-like
print("\n= Training LeNet-like CNN (we are using pre-trainned models) =")
# Train AlexNet-mini
print("\n= Training AlexNet-mini CNN (we are using pre-trainned models) =")


BaselineCNN parameters: 4,204,293
LeNetLikeCNN parameters: 4,227,653
AlexNetMini parameters: 2,652,645

= Training Baseline CNN (we are using pre-trainned models) =

= Training LeNet-like CNN (we are using pre-trainned models) =

= Training AlexNet-mini CNN (we are using pre-trainned models) =


**Parameter counts:**
  - Baseline: ~4.2M parameters
  - LeNet-like: ~4.2M (similar due to small extra conv layer)
  - AlexNet-mini: ~2.6M (deeper but uses smaller FC layers)

---

**✅ Code Task 5.4.5.1: Train for One Epoch (Quick Loop on Small Split)**

📘 Instruction</br>
Implement a minimal one-epoch training function for CPU:
- Name: CT_train_one_epoch(model, loader, criterion, optimizer, device='cpu') → (avg_loss, avg_acc)
- Accuracy: proportion of correct argmax predictions.
- Train only 1 epoch on CT_train_loader using CT_model.
- Use criterion = nn.CrossEntropyLoss() and optimizer = torch.optim.Adam(CT_model.parameters(), lr=1e-3) (or similar).
</br>Store: CT_train_loss1, CT_train_acc1 (floats) and print them.

In [22]:
import torch

CT_criterion = nn.CrossEntropyLoss()
CT_optimizer = torch.optim.Adam(CT_model.parameters(), lr=1e-3)

def CT_train_one_epoch(model, loader, criterion, optimizer, device='cpu'):
    model.train()
    running_loss, running_corrects, total = 0.0, 0, 0
    for CT_inputs, CT_labels in loader:
        CT_inputs, CT_labels = CT_inputs.to(device), CT_labels.to(device)
        optimizer.zero_grad()
        CT_outputs = model(CT_inputs)
        CT_loss = criterion(CT_outputs, CT_labels)
        CT_loss.backward()
        optimizer.step()

        running_loss += CT_loss.item() * CT_inputs.size(0)
        running_corrects += (CT_outputs.argmax(1) == CT_labels).sum().item()
        total += CT_labels.size(0)

    avg_loss = running_loss / total
    avg_acc  = running_corrects / total
    return avg_loss, avg_acc

CT_train_loss1, CT_train_acc1 = CT_train_one_epoch(CT_model, CT_train_loader, CT_criterion, CT_optimizer, device=device)
print("CT_train_loss1:", CT_train_loss1)
print("CT_train_acc1:", CT_train_acc1)


CT_train_loss1: 3.798859918894975
CT_train_acc1: 0.027853260869565216


---

### **6. Evaluation & Comparison**

We now evaluate and compare **all three trained architectures** on the **held-out test set**:

- **Baseline CNN (2 conv layers)**
- **LeNet-like CNN (3 conv layers)**
- **AlexNet-mini (5 conv layers)**

**Metrics for Comparison**

1. **Overall Test Accuracy**  
   - Quick benchmark for generalization.

2. **Parameter Count vs Accuracy**  
   - Does adding depth lead to proportional gains?  
   - Are deeper models *always* better?

Let’s compute these metrics for side-by-side comparison.


**Evaluation Function & Comparison**

In [24]:
import torch.optim as optim
lenet_optimizer = optim.Adam(
    params = lenet_model.parameters(), lr=1e-3
)

alex_optimizer = optim.Adam(
    params = alexmini_model.parameters(), lr=1e-3
)

criterion = nn.CrossEntropyLoss()

In [25]:
lenet_model, (train_losses, val_losses, train_accuracies, val_accuracies) = train_model(lenet_model, criterion, lenet_optimizer, train_loader, val_loader, num_epochs=5, device='cpu')

alexmini_model, (train_losses, val_losses, train_accuracies, val_accuracies) = train_model(alexmini_model, criterion, alex_optimizer, train_loader, val_loader, num_epochs=5, device='cpu')

Epoch 1/5
--------------------
Train Loss: 3.5379 Acc: 0.0605
Val Loss: 3.4483 Acc: 0.0720
Epoch 2/5
--------------------
Train Loss: 3.3130 Acc: 0.0944
Val Loss: 3.2641 Acc: 0.1046
Epoch 3/5
--------------------
Train Loss: 3.0499 Acc: 0.1532
Val Loss: 3.2077 Acc: 0.1182
Epoch 4/5
--------------------
Train Loss: 2.4925 Acc: 0.2945
Val Loss: 3.3526 Acc: 0.1413
Epoch 5/5
--------------------
Train Loss: 1.2638 Acc: 0.6321
Val Loss: 4.1060 Acc: 0.1264
Training complete in 9m 52s
Best val loss: 3.2077
Epoch 1/5
--------------------
Train Loss: 3.6131 Acc: 0.0234
Val Loss: 3.6123 Acc: 0.0245
Epoch 2/5
--------------------
Train Loss: 3.6117 Acc: 0.0251
Val Loss: 3.6135 Acc: 0.0177
Epoch 3/5
--------------------
Train Loss: 3.6114 Acc: 0.0282
Val Loss: 3.6145 Acc: 0.0177
Epoch 4/5
--------------------
Train Loss: 3.6111 Acc: 0.0275
Val Loss: 3.6153 Acc: 0.0177
Epoch 5/5
--------------------
Train Loss: 3.6110 Acc: 0.0251
Val Loss: 3.6162 Acc: 0.0177
Training complete in 16m 55s
Best val lo

In [23]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Reuse helper to compute accuracy
def evaluate_model(model, loader, device='cpu'):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    return acc

# Evaluate all three models
baseline_acc = evaluate_model(baseline_model, test_loader)
lenet_acc = evaluate_model(lenet_model, test_loader)
alexmini_acc = evaluate_model(alexmini_model, test_loader)

print(f"Baseline CNN Test Accuracy: {baseline_acc:.4f}")
print(f"LeNet-like CNN Test Accuracy: {lenet_acc:.4f}")
print(f"AlexNet-mini CNN Test Accuracy: {alexmini_acc:.4f}")

# Parameter counts
baseline_params = count_params(baseline_model)
lenet_params = count_params(lenet_model)
alexmini_params = count_params(alexmini_model)

# Compare parameter count vs accuracy
print("\nParameter vs Accuracy Comparison:")
print(f"Baseline: {baseline_params:,} params → {baseline_acc:.2%}")
print(f"LeNet-like: {lenet_params:,} params → {lenet_acc:.2%}")
print(f"AlexNet-mini: {alexmini_params:,} params → {alexmini_acc:.2%}")


Baseline CNN Test Accuracy: 0.0275
LeNet-like CNN Test Accuracy: 0.0294
AlexNet-mini CNN Test Accuracy: 0.0275

Parameter vs Accuracy Comparison:
Baseline: 4,204,293 params → 2.75%
LeNet-like: 4,227,653 params → 2.94%
AlexNet-mini: 2,652,645 params → 2.75%


#### **Reflection**

- **Overall Accuracy:**  
  - Baseline and LeNet-like perform similarly (~10–15%).  
  - AlexNet-mini struggles — deeper ≠ better when data is limited.

- **Parameter vs Accuracy Trade-off:**  
  - Despite having more layers, AlexNet-mini has *fewer* parameters (smaller FC layers) but still performs worse.  
  - LeNet-like adds depth without significant parameter penalty, slightly better generalization.

- **Learning Dynamics:**  
  - Training curves show **faster overfitting** in deeper networks.
  - Validation accuracy plateaued early — regularization (e.g., dropout, augmentation) is needed.

This sets the stage for **Section 7**, where we reflect conceptually on **depth vs generalization** and link findings to real-world architecture choices (LeNet, AlexNet, VGG).

---


**✅ Code Task 5.4.6.1: Quick Validation + Baseline vs Variant Comparison**

📘 Instruction</br>
Implement a simple evaluation function and compare BaselineCNN (no extra training here) vs CT_LeNetVariant (after your 1-epoch train).
- Write CT_evaluate(model, loader, device='cpu') → float_accuracy.
- Compute CT_val_acc_baseline and CT_val_acc_variant on CT_val_loader.
- Also compute CT_params_baseline and CT_params_variant using your CT_count_params.
- Store results in a dict CT_compare = {...} and print it.

In [26]:
def CT_evaluate(model, loader, device='cpu'):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for CT_imgs, CT_lbls in loader:
            CT_imgs, CT_lbls = CT_imgs.to(device), CT_lbls.to(device)
            CT_out = model(CT_imgs)
            CT_pred = CT_out.argmax(1)
            correct += (CT_pred == CT_lbls).sum().item()
            total += CT_lbls.size(0)
    return correct / total

CT_baseline_for_eval = BaselineCNN().to(device)   # fresh baseline
CT_params_baseline = CT_count_params(CT_baseline_for_eval)
CT_params_variant  = CT_count_params(CT_model)

CT_val_acc_baseline = CT_evaluate(CT_baseline_for_eval, CT_val_loader, device=device)
CT_val_acc_variant  = CT_evaluate(CT_model, CT_val_loader, device=device)

CT_compare = {
    "baseline_acc": CT_val_acc_baseline,
    "variant_acc":  CT_val_acc_variant,
    "params_baseline": CT_params_baseline,
    "params_variant":  CT_params_variant
}
print("CT_compare:", CT_compare)


CT_compare: {'baseline_acc': 0.020380434782608696, 'variant_acc': 0.02309782608695652, 'params_baseline': 4199205, 'params_variant': 4204069}


### **7. Reflection on Depth vs Generalization**

#### **Feature Hierarchy in CNNs**

- **Shallow CNNs (Baseline):**
  - First conv layer detects **edges and simple colors**.
  - Second layer combines edges into **basic textures** (e.g., fur patterns).

- **Deeper CNNs (LeNet-like, AlexNet-mini):**
  - Additional layers can capture **mid- and high-level features**:
    - **Edges → textures → object parts → full objects**
  - Theoretically better for **fine-grained classification** (e.g., 37 pet breeds).

  
**Does Deeper Always Mean Better?**

- **Our Results:**
  - LeNet-like slightly outperformed baseline (modest depth gain).
  - AlexNet-mini **did not improve** despite more layers:
    - Likely due to **limited training data** and **no augmentation**.
    - Deeper models require **more data** to learn effectively.

- **Overfitting Risk:**
  - Deeper models memorized training data quickly (high train accuracy).
  - Validation accuracy plateaued early → classic sign of **overfitting**.

**Parameter Efficiency vs Performance**

- Baseline vs LeNet-like: Similar parameter counts, slight accuracy boost.
- AlexNet-mini: More layers, **fewer fully connected parameters**, but weaker results → depth alone isn’t sufficient.

**Practical Considerations**

- **Training Cost:**  
  - More layers = more computation (even on CPU-only setup).  
  - Small gains may not justify cost unless dataset size increases.

- **Data Augmentation (NB03 Insight):**  
  - Crucial for enabling deeper models to generalize without overfitting.

- **Transfer Learning (Future Project):**  
  - Pretrained deeper networks (ResNet, VGG) can leverage learned features and outperform shallow models on small datasets.

**Takeaway**

Depth is **powerful but not magical**:  
- Without **enough data or regularization**, deeper CNNs fail to generalize.  
- Next steps (future notebooks) involve **augmentation + transfer learning** to unlock the full potential of deep architectures.
